Importing the Necessary Libraries and the links required

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(website_url,'lxml')

Processing the data using Beautifulsoup, and cleaning up the to achieve the first df output

In [3]:
My_table = soup.find('table',{'class':'wikitable sortable'})
A1= ""
A2= ""
A3= ""
i=0;
PostalCode=[]
temp_PostalCode=None
Borough=[]
temp_Borough=None
Neighbourhood=[]
temp_Neighbourhood=None
for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 3:
        A1 = cells[0].find(text=True)
        temp_PostalCode=str(A1)
        
        A2 = cells[1].findAll(text=True)
        temp_Borough=str(A2).replace(r"['","")
        temp_Borough=temp_Borough.replace(r"']","")
        temp_Borough=temp_Borough.replace("[\"","")
        temp_Borough=temp_Borough.replace("\"]","")
                  
        A3 = cells[2].find(text=True)     
        temp_Neighbourhood=str(A3).replace("\n","")
           
        
        PostalCode.append(temp_PostalCode)
        Borough.append(temp_Borough)
        Neighbourhood.append(temp_Neighbourhood)
        i=i+1
        #print('{} {} {} {}'.format(A1,A2,A3,i))        
df=pd.DataFrame()
df['PostalCode']=PostalCode
df['Borough']=Borough
df['Neighbourhood']=Neighbourhood

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


The functions used as an Aggregate function to combine the strings for duplicates and Neighbourhoodless boroughs

In [4]:
def custom_add(k):
    c=''
    for value in k:
        c=c+', '+value
    z=len(c)
    c=c[2:z]
    return c

def ignore_value(j):
    previous=''
    result=''
    for name in j:
        if(result.find(name)==-1):
            result=result+name
    return result

In [11]:
df.sort_values('PostalCode',inplace=True)
df=df.groupby('PostalCode').agg({'Borough':ignore_value,'Neighbourhood':custom_add},inplace=True).reset_index()
x,y=df.shape
for z in range(x):
    if (((df.iloc[z,1]!='Not assigned')) and (df.iloc[z,2]=='Not assigned')):
        df.iloc[z,2]=df.iloc[z,1]
k=(df['Borough']!="Not assigned")
df=df[k]
df.head(12)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


Shape of the dataframe

In [10]:
df.shape

(103, 3)